In [1]:
from src.utils import get_data, get_torch_sparse,get_score

corpus, queries, qrels  =get_data("scifact")
queries_ids=list(queries.keys())
document_ids= list(corpus.keys())

/opt/anaconda3/envs/rra/lib/python3.12/site-packages/beir/util.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
100%|██████████| 5183/5183 [00:00<00:00, 311590.95it/s]


In [2]:
from src.rra.retrievers import sparta
sparseModel= sparta.get_model()

In [3]:
import numpy as np
#please modify SPARTA in  beir.retrieval.models from np.int to int and from np.float to float
encoded_doc,encoded_q=sparta.encode(sparseModel,corpus,queries,qrels)
doc_sparse_torch,_= get_torch_sparse(encoded_doc,device="cpu")


query: 100%|██████████| 300/300 [00:00<00:00, 2366.12it/s]
/Users/arthur/Documents/reasearch/RRA/src/utils/utils.py:34: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  indices = torch.tensor([coo.row, coo.col], dtype=torch.int, device=device)


In [4]:
res,full_res=get_score(doc_sparse_torch,encoded_q,document_ids,queries_ids,qrels,top_k=100)
res

{'NDCG@1': 0.50333,
 'NDCG@10': 0.59784,
 'NDCG@100': 0.63118,
 'Recall@1': 0.48611,
 'Recall@10': 0.70428,
 'Recall@100': 0.859}

In [5]:
from src.rra.rsa import RSA
rsa_mat,Vd,Ut= RSA(alpha=2, iterations=1,device="cpu", batch_row=10,classic=True,).run(doc_sparse_torch)
rsa_score,rsa_full_res= get_score(rsa_mat,encoded_q,document_ids,queries_ids,qrels,Vd,Ut,top_k=100)
rsa_score

Literal Pistener...
Pragmatic Speaker...


100%|██████████| 3053/3053 [00:00<00:00, 4784.59it/s]


Pragmatic Listener...


100%|██████████| 3053/3053 [00:01<00:00, 2993.32it/s]


{'NDCG@1': 0.53,
 'NDCG@10': 0.62402,
 'NDCG@100': 0.65884,
 'Recall@1': 0.51278,
 'Recall@10': 0.73178,
 'Recall@100': 0.893}